## Dog v/s Cat Classifier

You can use this notebook to train our dog v/s cat classifier. FastAI makes it super easy to train the models too, under 10 lines of code we have a descent classifier!

In [ ]:
from fastai.vision.all import *
from pathlib import Path

In [ ]:
path = untar_data(URLs.PETS)
path.ls()

In [ ]:
files = get_image_files(path/"images")
len(files)

In [ ]:
def label_func(f): return str(f)[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, 
                                      label_func, 
                                      item_tfms=Resize(224))

In [ ]:
dls.show_batch()

As you can see we have the images ready to be loaded for training. We are going to use a resnet34 and finetune on top of that. Lets see what how it goes

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

In [ ]:
learn.predict(files[0])

In [ ]:
learn.show_results()

With just 2 epoch of training we have created a fairly descent classifier for dogs and cats, good on ya mate!

Now lets save it and generate the final model file. 

In [ ]:
learn.path

In [ ]:
learn.export()
print('Saved to: ', learn.path)

FastAI library usually saves the model in the `learn.path` folder. We will copy if from there to our CWD. 

In [ ]:
from shutil import copyfile
copyfile(learn.path/'export.pkl', 'model.pkl')

## Inference
Now lets figure out the about loading the models and running the inference. 

In [ ]:
os.listdir('images/')

In [ ]:
loaded_learner = load_learner('model.pkl')

In [ ]:
files = get_image_files('images')
dl = loaded_learner.dls.test_dl(files, item_tfms=Resize(224), num_workers=0)
dl

In [ ]:
files

In [ ]:
loaded_learner.predict(files[2])

## DataBlock tut

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)
path

In [ ]:
def label_func(fname):
    return 'cat' if fname.name[0].isupper() else 'dog'

In [ ]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y     = label_func,
                   item_tfms = Resize(224))

In [ ]:
dset = dblock.datasets(path/'images')
dset.train[0]

In [ ]:
dset.vocab

In [ ]:
dls = dblock.dataloaders(path/'images')
dls.show_batch()

In [ ]:
d